In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("JupyterHub-Spark").master("local[*]").config("spark.ui.port", "4042").getOrCreate()

print(spark.sparkContext.uiWebUrl)
spark

http://ceewater-cpu001.unity.rc.umass.edu:4042


In [7]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]
emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [8]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [11]:
# Show emp dataframe (ACTION)

emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [13]:
# Print Schema

emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [18]:
# Gender Fixed

from pyspark.sql.functions import when,col,expr

emp_gender_fixed=emp.withColumn("new_gender", when(col("gender")=='Male', 'M').when(col("gender")=='Female', 'F').otherwise(None))

#or

emp_gender_fixed1=emp.withColumn("new_gender", expr("case when gender= 'Male' then 'M' when gender= 'Female' then 'F' else null end"))

In [19]:
emp_gender_fixed1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         F|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|         M|
|        010|   

In [22]:
#Replace in String

from pyspark.sql.functions import regexp_replace

emp_name_fixed= emp_gender_fixed1.withColumn("new_name", regexp_replace(col("name"),"J","Z"))

In [23]:
emp_name_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|    Zill Wong|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|Zames Zohnson|
|        008|          102|     Kate Kim

In [35]:
#Convert Date
from pyspark.sql.functions import to_date

emp_date_fixed=emp_name_fixed.withColumn("hire_date", to_date(col("hire_date"), 'y-MM-dd'))

In [34]:
emp_date_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|    Zill Wong|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|Zames Zohnson|
|        008|          102|     Kate Kim

In [36]:
emp_date_fixed.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- new_gender: string (nullable = true)
 |-- new_name: string (nullable = true)



In [39]:
# Add Date Columns
# Add currrent date, current timestamp, extract year from hire_date

from pyspark.sql.functions import current_date, current_timestamp

emp_dated= emp_date_fixed.withColumn("date_now", current_date()).withColumn("timestamp_now", current_timestamp())

In [41]:
emp_dated.show(truncate=False)

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|employee_id|department_id|name         |age|gender|salary|hire_date |new_gender|new_name     |date_now  |timestamp_now             |
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|001        |101          |John Doe     |30 |Male  |50000 |2015-01-01|M         |Zohn Doe     |2025-02-15|2025-02-15 04:59:11.627201|
|002        |101          |Jane Smith   |25 |Female|45000 |2016-02-15|F         |Zane Smith   |2025-02-15|2025-02-15 04:59:11.627201|
|003        |102          |Bob Brown    |35 |Male  |55000 |2014-05-01|M         |Bob Brown    |2025-02-15|2025-02-15 04:59:11.627201|
|004        |102          |Alice Lee    |28 |Female|48000 |2017-09-30|F         |Alice Lee    |2025-02-15|2025-02-15 04:59:11.627201|
|005        |103          |Jack Chan    |40 |Male  |60000 |201

In [43]:
# Drop null values

emp_drop_null_values=emp_dated.na.drop()

In [44]:
emp_drop_null_values.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|2025-02-15|2025-02-15 05:00:...|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|2025-02-15|2025-02-15 05:00:...|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|2025-02-15|2025-02-15 05:00:...|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|2025-02-15|2025-02-15 05:00:...|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|2025-02-1

In [45]:
# fix null values
# select *, nvl('new_gender', 'o') as new_gender from emp_dated
from pyspark.sql.functions import coalesce, lit

emp_null_df=emp_dated.withColumn("new_gender", coalesce(col("new_gender"), lit("O")))

In [47]:
emp_null_df.show(truncate=False)

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|employee_id|department_id|name         |age|gender|salary|hire_date |new_gender|new_name     |date_now  |timestamp_now             |
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|001        |101          |John Doe     |30 |Male  |50000 |2015-01-01|M         |Zohn Doe     |2025-02-15|2025-02-15 05:04:09.951425|
|002        |101          |Jane Smith   |25 |Female|45000 |2016-02-15|F         |Zane Smith   |2025-02-15|2025-02-15 05:04:09.951425|
|003        |102          |Bob Brown    |35 |Male  |55000 |2014-05-01|M         |Bob Brown    |2025-02-15|2025-02-15 05:04:09.951425|
|004        |102          |Alice Lee    |28 |Female|48000 |2017-09-30|F         |Alice Lee    |2025-02-15|2025-02-15 05:04:09.951425|
|005        |103          |Jack Chan    |40 |Male  |60000 |201

In [48]:
# Drop old columns and add new columns

emp_final=emp_null_df.drop("name", "gender").withColumnRenamed("new_name", "name").withColumnRenamed("new_gender", "gender")

In [51]:
emp_final.show(truncate=False)

+-----------+-------------+---+------+----------+------+-------------+----------+--------------------------+
|employee_id|department_id|age|salary|hire_date |gender|name         |date_now  |timestamp_now             |
+-----------+-------------+---+------+----------+------+-------------+----------+--------------------------+
|001        |101          |30 |50000 |2015-01-01|M     |Zohn Doe     |2025-02-15|2025-02-15 05:06:46.861281|
|002        |101          |25 |45000 |2016-02-15|F     |Zane Smith   |2025-02-15|2025-02-15 05:06:46.861281|
|003        |102          |35 |55000 |2014-05-01|M     |Bob Brown    |2025-02-15|2025-02-15 05:06:46.861281|
|004        |102          |28 |48000 |2017-09-30|F     |Alice Lee    |2025-02-15|2025-02-15 05:06:46.861281|
|005        |103          |40 |60000 |2013-04-01|M     |Zack Chan    |2025-02-15|2025-02-15 05:06:46.861281|
|006        |103          |32 |52000 |2018-07-01|F     |Zill Wong    |2025-02-15|2025-02-15 05:06:46.861281|
|007        |101   

In [52]:
# Write DF into CSV

emp_final.write.format("csv").save("data/emp_final.csv")

In [53]:
# convert date into string and extract date information

from pyspark.sql.functions import date_format

emp_fixed=emp_final.withColumn("date_string", date_format(col("hire_date"), "dd/MM/yyyy"))

In [55]:
emp_fixed.show(truncate=False)

+-----------+-------------+---+------+----------+------+-------------+----------+--------------------------+-----------+
|employee_id|department_id|age|salary|hire_date |gender|name         |date_now  |timestamp_now             |date_string|
+-----------+-------------+---+------+----------+------+-------------+----------+--------------------------+-----------+
|001        |101          |30 |50000 |2015-01-01|M     |Zohn Doe     |2025-02-15|2025-02-15 05:10:39.297776|01/01/2015 |
|002        |101          |25 |45000 |2016-02-15|F     |Zane Smith   |2025-02-15|2025-02-15 05:10:39.297776|15/02/2016 |
|003        |102          |35 |55000 |2014-05-01|M     |Bob Brown    |2025-02-15|2025-02-15 05:10:39.297776|01/05/2014 |
|004        |102          |28 |48000 |2017-09-30|F     |Alice Lee    |2025-02-15|2025-02-15 05:10:39.297776|30/09/2017 |
|005        |103          |40 |60000 |2013-04-01|M     |Zack Chan    |2025-02-15|2025-02-15 05:10:39.297776|01/04/2013 |
|006        |103          |32 |5